In [58]:
import PyPDF2
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import os
GOLBAL_pages_number = 30

# ====================================== Scrapping a book that is text 
def extraction_text_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        
        # Initialize an empty string to store the extracted text
        extracted_text = ''
        number_of_pages = GOLBAL_pages_number                  ##

        # Loop through each page and extract the text
        for page_num in range(number_of_pages):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            extracted_text+=page_text
       
    return extracted_text

def find_start_and_end(extracted_text): 
    words_to_find_start = ["table of contents", "contents", "index"]
    words_to_find_end = ["chapter 1", "index", "preface", "Summary", "appendix"]
    start = []
    end = []
    for word in words_to_find_start:
        s = extracted_text.lower().find(word)
        if s != -1:
            start.append(s)
    for word in words_to_find_end :
        e = extracted_text.lower().find(word)
        if e != -1:
            end.append(e)
    start = sorted(start, reverse=False)
    end = sorted(end, reverse=False)
    return (start[0],end[0])


def roughly_table_of_content(all_extracted_text):
    start,end = find_start_and_end(all_extracted_text) 

    if start !=-1 and end != -1:
        modified_text = all_extracted_text[start:end]
        return modified_text
    else:
        return all_extracted_text
    
## ====================================== Scrapping a book that is images

def convert_pdf_to_img(pdf_file):
    images = convert_from_path(pdf_file, poppler_path=r"C:/Users/Mariam Barakat/Desktop/work/CIS_LLM/Git_hub/LLM_Research/Data Preprocessing/poppler/poppler-23.08.0/Library\bin", first_page=0, last_page=GOLBAL_pages_number  - 1)
    return images

def convert_image_to_text(file, place_of_Tesseract = 'C:/Users/Mariam Barakat/AppData/Local/Programs/Tesseract-OCR/tesseract.exe'):
    pytesseract.pytesseract.tesseract_cmd = place_of_Tesseract
    text = pytesseract.image_to_string(file)
    return text

def extraction_image_pdf(book_name):
    pdf_file = os.path.join( "..", "books", book_name)
    images = convert_pdf_to_img(pdf_file)
    final_text = ""
    for img in images:
        out = convert_image_to_text(img)
        final_text += out
    return final_text
## =============================================  Final scrapper

def pdf_to_text(book_name):
    pdf_path = os.path.join( "..", "books", book_name)
    all_extracted_text1 = extraction_text_pdf(pdf_path)
    filtered_text = roughly_table_of_content(all_extracted_text1)

    if (all_extracted_text1 == filtered_text):
        all_extracted_text2 = extraction_image_pdf(pdf_path)
        filtered_text = roughly_table_of_content(all_extracted_text2)
    return filtered_text

## ================================== Creating a prompt_template for the output 
def prompt_template(filtered_text):
    prompt_template = "{### Table of contents:" + filtered_text + "}"
    # Path to the text file
    file_path = "book_corpus.txt"

    # Open the file in append mode
    with open(file_path, 'a', encoding='utf-8') as file:
        # Write the string variable to the file
        file.write(prompt_template + '\n')  # Adding a newline character for clarity

## =================================== Loop through the directory to extract all books inside and add to one txt file



In [59]:
book_name = "book1.pdf"

In [60]:
extraction_image_pdf(book_name)

'Big data, machine learning, and more, using Python tools\n\nDavy Cielen\nArno D. B. Meysman\nMohamed Ali\n\ni MANNING\n\nIntroducing Data Science\nIntroducing\nData Science\n\nBIG DATA, MACHINE LEARNING,\nAND MORE, USING PYTHON TOOLS\n\nDAVY CIELEN\nARNO D. B. MEYSMAN\nMOHAMED ALI\n\nMANNING\nSHELTER ISLAND\nFor online information and ordering of this and other Manning books, please visit\nwww.manning.com. The publisher offers discounts on this book when ordered in quantity.\nFor more information, please contact\n\nSpecial Sales Department\nManning Publications Co.\n\n20 Baldwin Road\n\nPO Box 761\n\nShelter Island, NY 11964\nEmail: orders@manning.com\n\n©2016 by Manning Publications Co. All rights reserved.\n\nNo part of this publication may be reproduced, stored in a retrieval system, or transmitted, in\nany form or by means electronic, mechanical, photocopying, or otherwise, without prior written\npermission of the publisher.\n\nMany of the designations used by manufacturers and se

In [61]:
roughly_table_of_content(extraction_image_pdf(book_name))

'contents\n\n© MN DA KR WH RK\n\nData science in a big data world 1\n\nThe data science process 22\n\nMachine learning 57\n\nHandling large data on a single computer 85\nFirst steps in big data 119\n\nJoin the NoSQL movement 150\n\nThe rise of graph databases 190\n\nText mining and text analytics 218\n\nData visualization to the end user 253\ncontents\n\n'

In [62]:
print(pdf_to_text(book_name))

contents
1■Data science in a big data world 1
2■The data science process 22
3■Machine learning 57
4■Handling large data on a single computer 85
5■First steps in big data 119
6■Join the NoSQL movement 150
7■The rise of graph databases 190
8■Text mining and text analytics 218
9■Data visualization to the end user 253viicontents



In [63]:
t= pdf_to_text(book_name)

In [64]:
prompt_template(t)